In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Import widgets
from ipywidgets import widgets, interactive, interact
import ipywidgets as widgets
from IPython.display import display

from sklearn.ensemble import RandomForestRegressor

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
sales_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
calendar_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
submission_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
prices_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')


# ## Calculating WRMSSEE, from: https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

In [ ]:
from typing import Union

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm


class WRMSSEEvaluator(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 0  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'state_id',
            'store_id',
            'cat_id',
            'dept_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            'item_id',
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)[valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt)

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)

        all_scores = []
        for i, group_id in enumerate(self.group_ids):
            lv_scores = self.rmsse(valid_preds.groupby(group_id)[self.valid_target_columns].sum(), i + 1)
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, sort=False).prod(axis=1)
            all_scores.append(lv_scores.sum())

        return np.mean(all_scores)

In [ ]:
train_fold_df = sales_df.iloc[:, :-56]
valid_fold_df = sales_df.iloc[:, -28:]
evaluator = WRMSSEEvaluator(train_fold_df, valid_fold_df, calendar_df, prices_df)

## visualizing data series for different splits

In [ ]:
days = range(1 , 1941 + 1)
time_series_columns = [f'd_{i}' for i in days]

ids = np.random.choice(sales_df['id'].unique().tolist(), 1000)

series_ids = widgets.Dropdown(
    options=ids,
    value=ids[0],
    description='series_ids:'
)

def plot_data(series_ids):
    df = sales_df.loc[sales_df['id'] == series_ids][time_series_columns]
    df = pd.Series(df.values.flatten())

    df.plot(figsize=(20, 10), lw=2, marker='*')
    df.rolling(7).mean().plot(figsize=(20, 10), lw=2, marker='o', color='orange')
    plt.axhline(df.mean(), lw=3, color='red')
    plt.grid()

In [ ]:
w = interactive(
    plot_data,
    series_ids=series_ids
)
display(w)

In [ ]:
column_values1 = sales_df[['store_id']].values.ravel()
stores = pd.unique(column_values1)

column_values1 = sales_df[['dept_id']].values.ravel()
departments = pd.unique(column_values1)

print(stores)
print(departments)
for i in stores:
    
    df = sales_df.loc[(sales_df['dept_id'] == 'HOUSEHOLD_2') & (sales_df['store_id'] == i)][time_series_columns]
    
    df2 = df.sum()
    df2 = pd.Series(df2.values.flatten())
    
    ax = df2.rolling(7).mean().plot(figsize=(20, 10), lw=2)
    ax.set_xlabel("days passed")
    ax.set_ylabel("sum of sales")
    

In [ ]:

for i in departments:
    
    df = sales_df.loc[(sales_df['dept_id'] == i)][time_series_columns]
    df2 = df.sum()
    df2 = pd.Series(df2.values.flatten())
    df2.rolling(7).mean().plot(figsize=(20, 10), lw=2)
    
    ax.set_xlabel("days passed")
    ax.set_ylabel("sum of sales")

## changing datatypes, from: https://www.kaggle.com/ar2017/m5-forecasting-lightgbm

In [ ]:
# Correct data types for "calendar.csv"
calendarDTypes = {"event_name_1": "category", 
                  "event_name_2": "category", 
                  "event_type_1": "category", 
                  "event_type_2": "category", 
                  "weekday": "category", 
                  'wm_yr_wk': 'int16', 
                  "wday": "int16",
                  "month": "int16", 
                  "year": "int16", 
                  "snap_CA": "float32", 
                  'snap_TX': 'float32', 
                  'snap_WI': 'float32' }

# Read csv file
calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv", 
                       dtype = calendarDTypes)

calendar["date"] = pd.to_datetime(calendar["date"])

# Transform categorical features into integers
for col, colDType in calendarDTypes.items():
    if colDType == "category":
        calendar[col] = calendar[col].cat.codes.astype("int16")
        calendar[col] -= calendar[col].min()

calendar.head()

In [ ]:
firstDay = 1
lastDay = 1941

# Use x sales days (columns) for training
numCols = [f"d_{day}" for day in range(firstDay, lastDay+1)]

# Define all categorical columns
catCols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']

# Define the correct data types for "sales_train_validation.csv"
dtype = {numCol: "float32" for numCol in numCols} 
dtype.update({catCol: "category" for catCol in catCols if catCol != "id"})

# Read csv file
sales = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv", 
                 usecols = catCols + numCols, dtype = dtype)

# Transform categorical features into integers
for col in catCols:
    if col != "id":
        sales[col] = sales[col].cat.codes.astype("int16")
        sales[col] -= sales[col].min()
        


## Random forest

In [ ]:

features = calendar[{'year','snap_TX','snap_CA','wday','month', 'snap_WI', "event_name_1","event_type_1", "event_name_2","event_type_2"}].values


In [ ]:
#for each time series, take random indices. for each indices, take previous 14 days and calander data of day 15 as featuers and day 15 as y. 
# now this is done for each dept_id and each store, but can also be done for diferent splits. 

column_values1 = sales[['store_id']].values.ravel()
stores = pd.unique(column_values1)

column_values1 = sales[['dept_id']].values.ravel()
departments = pd.unique(column_values1)

days = range(1, 1941 + 1)

first_day = 1

time_series_columns = [f'd_{i}' for i in days]


predict_validation = np.zeros((30490, 28))
predict_evaluation = np.zeros((30490, 28))


for storeid in stores:
    for deptid in departments:


        itemseries2 = sales.loc[(sales['dept_id'] == deptid ) & (sales['store_id'] == storeid)]
        indexen = itemseries2.index.values.tolist()
        itemseries = itemseries2[time_series_columns].values



        n1 = 100 #number of samplse per time series
        xo =  1
        xi = 14
        train_x_data = np.zeros((len(itemseries)* n1,xi+len(features[0])))
        train_y_data =  np.zeros(len(train_x_data))

        # for loop creating the samples.
        for i in range(len(itemseries)):
            x = itemseries[i]

            n_random = np.random.choice(  np.arange(first_day + 60 ,len(x)-84), n1, replace = False)
        
            x_train = np.zeros((n1,xi+len(features[0])))
            y_train = np.zeros(len(x_train))

            for k, j in enumerate(n_random):
                x_train[k] = np.concatenate((x[j -xi-xo:j-1], features[j-1]), axis = 0) 
                y_train[k] = x[j-1]

            train_x_data[i*n1:i*n1+n1] = x_train
            train_y_data[i*n1:i*n1+n1] = y_train

        rf = RandomForestRegressor(n_estimators = 100, random_state = 4)# Train the model on training data
        rf.fit(train_x_data,train_y_data )





        ## predicting the validation data: days 1914 - 1941

        x_predict = itemseries[:,-14-28:-28] #initial last 14 days for day 1914

        predict_features = features[-56:-28] # calander data of the 28 days to predict

        # concatenating the above 2 variables
        predict_total = np.zeros((len(x_predict), len(x_predict[0])+len(predict_features[0])))
        for z in range(len(x_predict)):
            predict_total[z] = np.concatenate((x_predict[z],predict_features[0]), axis= 0)

        #predicting the first day for every time series
        initial_predict = rf.predict(predict_total)


        #this for loop takes the previous last 13 days of a predicion and add the prediction of the previous day as the 14th day. 
        # it then adds the calandar data of that day as well.
        # the prediction is added to predict_total2 and then the loop is repeated. 
        predict_total2 = np.zeros((len(x_predict),28))
        temp_prediction = initial_predict
        predict_total2[:,0] = temp_prediction
        to_predict = x_predict.astype(np.float32)
        for i in range(1,28):
           
            to_predict[:,:13] = to_predict[:,-13:]
            to_predict[:,13] = temp_prediction

            predict_total = np.zeros((len(x_predict), len(x_predict[0])+len(predict_features[0])))
            for j in range(len(x_predict)):
                predict_total[j] = np.concatenate((to_predict[j],predict_features[i]), axis= 0)

            temp_prediction = rf.predict(predict_total)
            predict_total2[:,i] = temp_prediction

       
        predict_validation[indexen] = predict_total2


    
    
    
    ## predicting the evaluation data: days 1942 - 1969
    
    x_predict = itemseries[:,-14:] #initial last 14 days for day 1914
    
    predict_features = features[-28:] # calander data of the 28 days to predict

    # concatenating the above 2 variables
    predict_total = np.zeros((len(x_predict), len(x_predict[0])+len(predict_features[0])))
    for z in range(len(x_predict)):
        predict_total[z] = np.concatenate((x_predict[z],predict_features[0]), axis= 0)

    #predictin the first day for every time series
    initial_predict = rf.predict(predict_total)


    #this forloop takes the previous last 13 days of a predicion and add the prediction of the previous day as the 14th day. 
    # it then adds the calandar data of that day as well.
    # the prediction is added to predict_total2 and then the loop is repeated. 
    predict_total2 = np.zeros((len(x_predict),28))
    temp_prediction = initial_predict
    predict_total2[:,0] = temp_prediction
    to_predict = x_predict.astype(np.float32)
    for i in range(1,28):

        to_predict[:,:13] = to_predict[:,-13:]
        to_predict[:,13] = temp_prediction

        predict_total = np.zeros((len(x_predict), len(x_predict[0])+len(predict_features[0])))
        for j in range(len(x_predict)):
            predict_total[j] = np.concatenate((to_predict[j],predict_features[i]), axis= 0)

  
        temp_prediction = rf.predict(predict_total)
        predict_total2[:,i] = temp_prediction
  
    predict_evaluation[indexen] = predict_total2

## next cell was used if rolling means and lags were the features

In [ ]:

'''
#for each time series, take 10 random indices. for each indices, take previous 14 days and calander data of day 15 as featuers and day 15 as y. 
# now this is done for each dept_id

#for itemid in unique_values1:
#    itemseries = train_sales.loc[(train_sales['dept_id'] == itemid)]
#    print(len(itemseries))

column_values1 = sales[['store_id']].values.ravel()
stores = pd.unique(column_values1)

column_values1 = sales[['dept_id']].values.ravel()
departments = pd.unique(column_values1)



days = range(1, 1941 + 1)
val_days = range(1914, 1941+1)
first_day = 1

time_series_columns = [f'd_{i}' for i in days]
val_days_columns = [f'd_{i}' for i in val_days]

# shape is number of series, number of predicted days
predict_total3 = np.zeros((30490, 56))



#for storeid in stores:
for deptid in departments:
    

    itemseries2 = sales.loc[(sales['dept_id'] == deptid)] # & (sales['store_id'] == storeid)
    indexen = itemseries2.index.values.tolist()
    itemseries = itemseries2[time_series_columns].values



    n1 = 100 #100  #number of samplse per time series
    xo =  1
    xi = 6
    train_x_data = np.zeros((len(itemseries)* n1,xi+len(features[0])))
    train_y_data =  np.zeros(len(train_x_data))


    for i in range(len(itemseries)):
        x = itemseries[i]

        
        n_random = np.random.choice( np.arange(first_day + 60 ,len(x)-84), n1, replace = False)

        x_train = np.zeros((n1,xi+len(features[0])))
        y_train = np.zeros(len(x_train))

        #creating training  samples
        for k, j in enumerate(n_random):
            lag_7 = x[j-8]
            lag_28 = x[j-29]
            rmean_7_7 = np.mean(x[j-15:j-8])
            rmean_28_7 = np.mean(x[j-36:j-29])
            rmean_7_28 = np.mean(x[j-36:j-8])
            rmean_28_28 = np.mean(x[j-57:j-29])
            means = np.array([lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28])
            x_train[k] = np.concatenate((means, features[j-1]), axis = 0) 
            y_train[k] = x[j-1]
       


        train_x_data[i*n1:i*n1+n1] = x_train
        train_y_data[i*n1:i*n1+n1] = y_train

    rf = RandomForestRegressor(n_estimators = 100, random_state = 4)# Train the model on training data
    rf.fit(train_x_data,train_y_data )

    predict_features = features[-56:] # calander data of the 56 days to predict


    predict_total2 = np.zeros((len(itemseries),56))


    k = itemseries[:,:-28] 
    q = itemseries

    
    #making predictions for 56 days (28 validation and 28 for submission)
    for i in range(56):
        if i < 28:
            


            lag_7 = k[:,-7]
            lag_28 = k[:,-28]
            rmean_7_7 = np.mean(k[:,-14:-7],axis=1)
            rmean_28_7 = np.mean(k[:,-35:-28],axis=1)
            rmean_7_28 = np.mean(k[:,-35:-7],axis=1)
            rmean_28_28 = np.mean(k[:,-56:-28],axis=1)
            means = np.column_stack((lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28)) #np.array([lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28])

            predict_total = np.zeros((len(itemseries), len(means[0])+len(predict_features[0])))
            for z in range(len(itemseries)):
                predict_total[z] = np.concatenate((means[z],predict_features[i]), axis= 0)

            predict = rf.predict(predict_total)
            k = np.column_stack((k, predict))
          
            predict_total2[:,i] = predict

        else:

            lag_7 = q[:,-7]
            lag_28 = q[:,-28]
            rmean_7_7 = np.mean(q[:,-14:-7],axis=1)
            rmean_28_7 = np.mean(q[:,-35:-28],axis=1)
            rmean_7_28 = np.mean(q[:,-35:-7],axis=1)
            rmean_28_28 = np.mean(q[:,-56:-28],axis=1)
            means = np.column_stack((lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28)) #np.array([lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28])

            predict_total = np.zeros((len(itemseries), len(means[0])+len(predict_features[0])))
            for z in range(len(itemseries)):
                predict_total[z] = np.concatenate((means[z],predict_features[i]), axis= 0)

            predict = rf.predict(predict_total)
            q = np.column_stack((q, predict))
    
            predict_total2[:,i] = predict

    predict_total3[indexen] = predict_total2




predict_validation = predict_total3[:,:-28]
predict_evaluation = predict_total3[:,-28:]

'''

## WRMSSE on validation set (1914-1941)

In [ ]:
val_days = range(1914, 1941+1)
val_days_columns = [f'd_{i}' for i in val_days]

valid_preds2 = pd.DataFrame(predict_validation ,columns = val_days_columns)

# getting error of 28 validation days
print(evaluator.score(valid_preds2))

# Predictions


In [ ]:
forecast_val = pd.DataFrame(predict_validation)
forecast_eval = pd.DataFrame(predict_evaluation)

In [ ]:
forecast_val.columns = [f'F{i}' for i in range(1, forecast_val.shape[1] + 1)]
forecast_eval.columns = [f'F{i}' for i in range(1, forecast_eval.shape[1] + 1)]

In [ ]:
evaluation_ids  = sales_df['id'].values
validation_ids = [i.replace('evaluation', 'validation') for i in evaluation_ids]

In [ ]:
ids = np.concatenate([validation_ids, evaluation_ids])

In [ ]:
predictions = pd.DataFrame(ids, columns=['id'])

In [ ]:
forecast = pd.concat([forecast_val, forecast_eval]).reset_index(drop=True)


In [ ]:
final_rf =  pd.concat([predictions, forecast], axis=1)

In [ ]:
final_rf

In [ ]:
final_rf.shape

In [ ]:
final_rf.to_csv('submission.csv', index=False)